In [ ]:
#!unzip "/content/drive/My Drive/NLP_IMDBREVIEW/9914_14036_compressed_imdb_master.csv.zip" -d "/content/drive/My Drive/NLP_IMDBREVIEW/"

Archive:  /content/drive/My Drive/NLP_IMDBREVIEW/9914_14036_compressed_imdb_master.csv.zip
  inflating: /content/drive/My Drive/NLP_IMDBREVIEW/imdb_master.csv  


In [ ]:
!unzip "/content/drive/My Drive/NLP_IMDBREVIEW/labeledTrainData.tsv.zip" -d "/content/drive/My Drive/NLP_IMDBREVIEW/"

Archive:  /content/drive/My Drive/NLP_IMDBREVIEW/labeledTrainData.tsv.zip
  inflating: /content/drive/My Drive/NLP_IMDBREVIEW/labeledTrainData.tsv  


In [ ]:
#review=pd.read_csv("/content/drive/My Drive/NLP_IMDBREVIEW/labeledTrainData.tsv",header=0, delimiter="\t", quoting=3)
#review.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
#review.tail()

,id,sentiment,review
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."
24999,"""8478_8""",1,"""I saw this movie as a child and it broke my h..."


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [ ]:
# Importing dataset into notebook
# we need to encode this file some of the data can't be decoded by UTf-8 , hence used "latin-1" encoding
# quoting=2 means if there are any "" double quotes in the csv file, mainly in the review column, the pd.read_csv will ignore those ""
review = pd.read_csv("/content/drive/My Drive/NLP_IMDBREVIEW/imdb_master.csv", encoding="latin-1", index_col=0)
review.head()

,type,review,label,file
0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [ ]:
review.label.value_counts()

unsup    50000
neg      25000
pos      25000
Name: label, dtype: int64

In [ ]:
review = review[review.label != 'unsup']

In [ ]:
review.label.value_counts()

neg    25000
pos    25000
Name: label, dtype: int64

In [ ]:
review.shape

(50000, 4)

In [ ]:
dataset = review[['review', 'label']]
dataset.head()

,review,label
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [ ]:
dataset.shape

(50000, 2)

###Split the data into training and testing

In [ ]:
# training = dataset[:80000]
# testing = dataset.review[80000:]

In [ ]:
# training.shape, testing.shape

((80000, 2), (20000,))

### Cleaning the texts

In [ ]:
import re
# nltk is a library which ensemble "stopwords", we need to remove the stopwords from our review, hence they are of no use in predicting whether review is positive or negative
# All articles, "the", "a", "an" are stopwords, 'he','she'
import nltk
# Now we will download the stopwords from nltk library
nltk.download('stopwords')
# Now we are importing all downloaded stopwrds into our jupyter notebook
from nltk.corpus import stopwords
# Importing PortStemmer class in order to apply Stemming in our text
# Stemming means taking a root of the words which allows us to get the true meaning of the word eg. "loved" and "love", both have the same meaning (positive), hence model should be able to predict the true
# meaning although we use the word in different tense. Stemming allows us to do that. It allows to convert all words into present tense.
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# creating a function to remove punctuation or non letters words
def remove_punctuation(text):
  """
  Removing punctuation from the text
  """
  review = re.sub('[^a-zA-Z]', ' ', text)

  return review

In [ ]:
dataset['Processed_Reviews'] = dataset.review.apply(lambda x: remove_punctuation(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
## Creating a function to convert all words into lowercase
def lowercase(text):
  """
  Converting text into lowercase
  """

  review = text.lower()

  return review

In [ ]:
dataset['Processed_Reviews'] =dataset.Processed_Reviews.apply(lambda x: lowercase(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ps = PorterStemmer()

In [ ]:
def text_stemming(text):
  """
  this function does the stemming of the text and remove stopwords
  """
  review = text.split()
  # ps = PorterStemmer()
  # applying stemming for all words for each row in review and removing stopwords from the reviews
  
  # review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)

  return review

In [ ]:
for word in set(stopwords.words('english')):
    print(word)

### It took around 50 minutes for text_stemming() function to get the stemming done for the whole dataset😲

In [ ]:
dataset['Processed_Reviews'] =dataset.Processed_Reviews.apply(lambda x: text_stemming(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# LabelEncoding for label column 

In [ ]:
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
dataset['label_encoding'] = labelencoder.fit_transform(dataset['label'])

In [ ]:
dataset.head()

,review,label,Processed_Reviews,label_encoding
0,Once again Mr. Costner has dragged out a movie...,neg,mr costner drag movi far longer necessari asid...,0
1,This is an example of why the majority of acti...,neg,exampl major action film gener bore realli not...,0
2,"First of all I hate those moronic rappers, who...",neg,first hate moron rapper could nt act gun press...,0
3,Not even the Beatles could write songs everyon...,neg,not even beatl could write song everyon like a...,0
4,Brass pictures (movies is not a fitting word f...,neg,brass pictur movi not fit word realli somewhat...,0


In [ ]:
dataset.to_csv('/content/drive/My Drive/NLP_IMDBREVIEW/imdb_processed_review4.csv')

In [ ]:
df = pd.read_csv("/content/drive/My Drive/NLP_IMDBREVIEW/imdb_processed_review4.csv")

In [ ]:
df.head()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
0,0,Once again Mr. Costner has dragged out a movie...,neg,mr costner drag movi far longer necessari asid...,0
1,1,This is an example of why the majority of acti...,neg,exampl major action film gener bore realli not...,0
2,2,"First of all I hate those moronic rappers, who...",neg,first hate moron rapper could nt act gun press...,0
3,3,Not even the Beatles could write songs everyon...,neg,not even beatl could write song everyon like a...,0
4,4,Brass pictures (movies is not a fitting word f...,neg,brass pictur movi not fit word realli somewhat...,0


In [ ]:
df.tail()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
49995,49995,"Seeing as the vote average was pretty low, and...",pos,see vote averag pretti low fact clerk video st...,1
49996,49996,"The plot had some wretched, unbelievable twist...",pos,plot wretch unbeliev twist howev chemistri mel...,1
49997,49997,I am amazed at how this movie(and most others ...,pos,amaz movi other averag star lower crappi movi ...,1
49998,49998,A Christmas Together actually came before my t...,pos,christma togeth actual came time rais john den...,1
49999,49999,Working-class romantic drama from director Mar...,pos,work class romant drama director martin ritt u...,1


In [ ]:
# Shuffling the data
from sklearn.utils import shuffle
df = shuffle(df)
df.head()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
40558,40558,"The late, great Robert Bloch (author of PSYCHO...",pos,late great robert bloch author psycho pay atte...,1
47180,47180,It does seem like this film is polarizing us. ...,pos,seem like film polar us either love hate love ...,1
45302,45302,Since I am so interested in lake monsters i re...,pos,sinc interest lake monster realli dug movi mov...,1
17966,17966,"Yes, at times ""Unconditional Love"" overwhelms ...",pos,ye time uncondit love overwhelm bounc ricochet...,1
41450,41450,I thought this was a very good movie. It would...,pos,thought good movi would nice seen littl movi s...,1


In [ ]:
df.tail()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
14489,14489,I really liked this movie. If other people wan...,pos,realli like movi peopl want give averag around...,1
12857,12857,This is the very first Three Stooges short wit...,pos,first three stoog short shemp saw one favorit ...,1
11601,11601,One of the worst movies ever made. Let's start...,neg,one worst movi ever made let start superfici j...,0
43641,43641,This is the movie I've seen more times than an...,pos,movi seen time believ saw averag everi year si...,1
22424,22424,I've been waiting 30 years to see this film. I...,pos,wait year see film play soundtrack album teena...,1


In [ ]:
#resetting the index
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
0,40558,"The late, great Robert Bloch (author of PSYCHO...",pos,late great robert bloch author psycho pay atte...,1
1,47180,It does seem like this film is polarizing us. ...,pos,seem like film polar us either love hate love ...,1
2,45302,Since I am so interested in lake monsters i re...,pos,sinc interest lake monster realli dug movi mov...,1
3,17966,"Yes, at times ""Unconditional Love"" overwhelms ...",pos,ye time uncondit love overwhelm bounc ricochet...,1
4,41450,I thought this was a very good movie. It would...,pos,thought good movi would nice seen littl movi s...,1


In [ ]:
df.shape

(50000, 5)

### Splitting full dataset into training and test dataset

In [ ]:
training = df[:40000]
testing = df[40000:]

In [ ]:
testing.head()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
40000,31742,This is a typical low budget 1970's mess. It's...,neg,typic low budget mess suppos docudrama crew hu...,0
40001,28939,"""Back of Beyond"" takes place at a dive diner/g...",neg,back beyond take place dive diner ga station m...,0
40002,49489,"Wow baby, this is indeed some fine Asian horro...",pos,wow babi inde fine asian horror gore crazi out...,1
40003,2987,"Oh, where the hell should I begin? Give a brie...",neg,oh hell begin give brief summari stori either ...,0
40004,11244,Tu pa tam is one of the worst movies I've ever...,neg,tu pa tam one worst movi ever seen hand better...,0


In [ ]:
testing.reset_index(drop=True, inplace=True)

In [ ]:
training.head()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
0,40558,"The late, great Robert Bloch (author of PSYCHO...",pos,late great robert bloch author psycho pay atte...,1
1,47180,It does seem like this film is polarizing us. ...,pos,seem like film polar us either love hate love ...,1
2,45302,Since I am so interested in lake monsters i re...,pos,sinc interest lake monster realli dug movi mov...,1
3,17966,"Yes, at times ""Unconditional Love"" overwhelms ...",pos,ye time uncondit love overwhelm bounc ricochet...,1
4,41450,I thought this was a very good movie. It would...,pos,thought good movi would nice seen littl movi s...,1


### Tokenizing and Converting words into integer.

This is done for training data below

In [ ]:
from keras.preprocessing.text import Tokenizer
max_features = 5000
#num_words means that we want to keep most frequent 700 words from the from each row from the dataset
tokenizer = Tokenizer(num_words=max_features)
# fit function will provide integer token to each word in the Processed Review column, and assign the value only once to the words irrespective of its occurrence
tokenizer.fit_on_texts(training['Processed_Reviews'])
# text_to_sequences create sequence of most frequent
list_tokenized_train = tokenizer.texts_to_sequences(training['Processed_Reviews'])

In [ ]:
print('Num words = {0}'.format(len(tokenizer.word_index)))

Num words = 61417


### Tokenizer and Converting words into integers

This is done for test data below

In [ ]:
testing.head()

,Unnamed: 0,review,label,Processed_Reviews,label_encoding
40000,40000,Kate Beckinsale steals the show! Bravo! Too ba...,pos,kate beckinsal steal show bravo bad knightli i...,1
40001,40001,It has taken me a while to watch this version ...,pos,taken watch version unfortun seem abl rent vid...,1
40002,40002,"I've really enjoyed this adaptation of ""Emma""....",pos,realli enjoy adapt emma seen mani time alway l...,1
40003,40003,This has long been one of my favourite adaptat...,pos,long one favourit adapt austen novel although ...,1
40004,40004,I've seen this movie after watching Paltrow's ...,pos,seen movi watch paltrow version found one good...,1


In [ ]:
from keras.preprocessing.text import Tokenizer
#num_words means that we want to keep most frequent 700 words from the from each row from the dataset
#tokenizer = Tokenizer(num_words=max_features)
# fit function will provide integer token to each word in the Processed Review column, and assign the value only once to the words irrespective of its occurrence
#tokenizer.fit_on_texts(testing['Processed_Reviews'])
# text_to_sequences create sequence of most frequent
list_tokenized_test = tokenizer.texts_to_sequences(testing['Processed_Reviews'])

# Padding 0 into the testing and training data.

This will make sure that all review length are same.

In [ ]:
import tensorflow as tf
X_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train,maxlen=130)

In [ ]:
y_train = training['label_encoding']

In [ ]:
y_train.shape

(40000,)

In [ ]:
len(X_train)

40000

### Padding testing data

In [ ]:
import tensorflow as tf
X_test = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_test,maxlen=130)

Getting labels for testing data

In [ ]:
y_test = testing['label_encoding']

In [ ]:
y_test.shape

(10000,)

### Building the model

In [ ]:
X_train.shape

(40000, 638)

In [ ]:
print(min(600, 1.6*40000**0.56))
print(40000**0.25)

600
14.142135623730951


In [ ]:
print(max_features)

5000


In [ ]:
125*40000

5000000

In [ ]:
# This will give you average number of words per row, you can decide output dim according to this
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

123.40708

In [ ]:
testing['label_encoding'].value_counts()

1    10000
Name: label_encoding, dtype: int64

In [ ]:
# X_train_seq = pd.Series(X_train)
# X_train_len = X_train_seq.map(lambda ls: len(ls))
# X_train_len.describe()

In [ ]:
## Early stopping callback
import tensorflow as tf
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience = 3)

In [ ]:
NUM_EPOCHS = 100

In [ ]:
rnn_model = tf.keras.Sequential()
# input dimension should be the vocalbulary size which in our case is max_features which is 700
# Ouptput dimension should be around formula min(600, 1.6*cardinality**0.56)
rnn_model.add(tf.keras.layers.Embedding(input_dim=max_features, output_dim=130, input_shape=(X_train.shape[1], )))
rnn_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128, return_sequences=True,activation='tanh')))
#rnn_model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))
rnn_model.add(tf.keras.layers.GlobalMaxPool1D())
#rnn_model.add(tf.keras.layers.Flatten())
rnn_model.add(tf.keras.layers.Dense(units=30, activation='relu'))
rnn_model.add(tf.keras.layers.Dropout(0.5))
rnn_model.add(tf.keras.layers.Dense(units=10, activation='relu'))
rnn_model.add(tf.keras.layers.Dropout(0.5))
rnn_model.add(tf.keras.layers.Dropout(0.5))
rnn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
rnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 130, 130)          650000    
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 130, 256)          265216    
_________________________________________________________________
global_max_pooling1d_14 (Glo (None, 256)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 30)                7710      
_________________________________________________________________
dropout_47 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 10)                310       
_________________________________________________________________
dropout_48 (Dropout)         (None, 10)              

In [ ]:
validation_split = 0.01
rnn_model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=512, validation_split = validation_split, callbacks = [early_stopping])

Epoch 1/100
78/78 [==============================] - 7s 87ms/step - loss: 0.6527 - accuracy: 0.5947 - val_loss: 0.4469 - val_accuracy: 0.8550
Epoch 2/100
78/78 [==============================] - 6s 80ms/step - loss: 0.5145 - accuracy: 0.7552 - val_loss: 0.3576 - val_accuracy: 0.8925
Epoch 3/100
78/78 [==============================] - 6s 80ms/step - loss: 0.4481 - accuracy: 0.8096 - val_loss: 0.2998 - val_accuracy: 0.8850
Epoch 4/100
78/78 [==============================] - 6s 81ms/step - loss: 0.4138 - accuracy: 0.8281 - val_loss: 0.2965 - val_accuracy: 0.8975
Epoch 5/100
78/78 [==============================] - 6s 81ms/step - loss: 0.3854 - accuracy: 0.8443 - val_loss: 0.2970 - val_accuracy: 0.9000
Epoch 6/100
78/78 [==============================] - 6s 81ms/step - loss: 0.3720 - accuracy: 0.8468 - val_loss: 0.3085 - val_accuracy: 0.8750
Epoch 7/100
78/78 [==============================] - 6s 81ms/step - loss: 0.3544 - accuracy: 0.8542 - val_loss: 0.2792 - val_accuracy: 0.9000
Epoch 

### Working on test data

In [ ]:
!unzip "/content/drive/My Drive/NLP_IMDBREVIEW/testData.tsv.zip" -d "/content/drive/My Drive/NLP_IMDBREVIEW/"

Archive:  /content/drive/My Drive/NLP_IMDBREVIEW/testData.tsv.zip
  inflating: /content/drive/My Drive/NLP_IMDBREVIEW/testData.tsv  


In [ ]:
df_test=pd.read_csv("/content/drive/My Drive/NLP_IMDBREVIEW/testData.tsv",header=0, delimiter="\t", quoting=3)
df_test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [ ]:
df_test.shape

(25000, 2)

In [ ]:
df_test['Processed_Reviews'] = df_test.review.apply(lambda x: remove_punctuation(x))
df_test['Processed_Reviews'] = df_test.Processed_Reviews.apply(lambda x: lowercase(x))

In [ ]:
df_test['Processed_Reviews'] =df_test.Processed_Reviews.apply(lambda x: text_stemming(x))

In [ ]:
df_test.head()

,id,review,Processed_Reviews
0,"""12311_10""","""Naturally in a film who's main themes are of ...",natur film main theme mortal nostalgia loss in...
1,"""8348_2""","""This movie is a disaster within a disaster fi...",movi disast within disast film full great acti...
2,"""5828_4""","""All in all, this is a movie for kids. We saw ...",movi kid saw tonight child love one point kid ...
3,"""7186_2""","""Afraid of the Dark left me with the impressio...",afraid dark left impress sever differ screenpl...
4,"""12128_7""","""A very accurate depiction of small time mob l...",accur depict small time mob life film new jers...


In [ ]:
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
list_tokenized_test = tokenizer.texts_to_sequences(df_test['Processed_Reviews'])
X_test = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_test,maxlen=130)

In [ ]:
prediction = rnn_model.predict(X_test)

In [ ]:
y_pred = (prediction >=0.7)*1

In [ ]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])